# Unemployment rate at count level from year 1990-2024
Amy Chung
~~~
The data comes from "LOCAL AREA UNEMPLOYMENT STATISTICS (LA)" from U.S. Bureau of Labor Statistics.
A detailed description of the database can be found in the following link:
https://download.bls.gov/pub/time.series/la/la.txt
~~~

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import json
from io import StringIO
import pickle
import sys

def crawl_data(main_url):
    """
    crawl data from a URL and organize it into a pandas DataFrame
    Input: main_url - a string of url
    Return: pandas DataFrame
    """

    result = requests.get(main_url)

    # extract data and organize a row as a list
    soup = BeautifulSoup(result.text, 'lxml')
    table = soup.find_all('body')[0]
    data = str(table)
    data = data.split('<body>')[1].split('</body>')[0].split('<p>')[1].split('</p>')[0]
    rows = data.split('\n')

    # split a row so that they could transorm into pandas DataFrame
    output = []
    for r in rows:
        row_ls = r.split("\t")
        for i in range(len(row_ls)):
            row_ls[i] = row_ls[i].strip()
        output.append(row_ls)
    df = pd.DataFrame(output[1:], columns=output[0])

    return df


def split_string(x):
    """
    split a string with "," and return the second element, if there is no "," in the string, return x
    Input: a string
    Return: a string
    """
    try:
        return x.split(",")[1].strip()
    except:
        return x

def unemployment_rate_tojson():
    """
    scrape all enunemployment rate data from year 1990 to 2024, organize into a json object
    Return: a json object
    """
    # dictionary for mapping state names
    us_state = {'AL': 'Alabama',
        'AK': 'Alaska',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'AR': 'Arkansas',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DE': 'Delaware',
        'DC': 'District of Columbia',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'IA': 'Iowa',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'ME': 'Maine',
        'MD': 'Maryland',
        'MA': 'Massachusetts',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MS': 'Mississippi',
        'MO': 'Missouri',
        'MT': 'Montana',
        'NE': 'Nebraska',
        'NV': 'Nevada',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NY': 'New York',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'MP': 'Northern Mariana Islands',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VT': 'Vermont',
        'VI': 'Virgin Islands',
        'VA': 'Virginia',
        'WA': 'Washington',
        'WV': 'West Virginia',
        'WI': 'Wisconsin',
        'WY': 'Wyoming'}


    # collect unemployment rate from 1990 to 2024
    time = ["00-04", "05-09", "10-14", "15-19", "20-24"]
    url_dict = {"00-04": "https://download.bls.gov/pub/time.series/la/la.data.0.CurrentU00-04", 
        "05-09": "https://download.bls.gov/pub/time.series/la/la.data.0.CurrentU05-09",
        "10-14": "https://download.bls.gov/pub/time.series/la/la.data.0.CurrentU10-14",
        "15-19": "https://download.bls.gov/pub/time.series/la/la.data.0.CurrentU15-19",
        "20-24": "https://download.bls.gov/pub/time.series/la/la.data.0.CurrentU20-24",
        "area": "https://download.bls.gov/pub/time.series/la/la.area"}

    # for each URL, crawl to a pandas dataframe and save the dataframe in a dictionary
    df_dict = {}
    for url in url_dict:
        df = crawl_data(url_dict[url])
        df_dict[url] = df


    # append employment data from 1990 to 2024 into one single big dataframe
    all_data = pd.DataFrame()
    for t in time:
        all_data = all_data.append(df_dict[t], ignore_index=True)

    # extract area_code and measure_code from series_id
    all_data["area_code"] = all_data["series_id"].apply(lambda x: x[3:-2])
    all_data["measure_code"] = all_data["series_id"].apply(lambda x: x[-2:])
    return all_data

### Scrape the original data and combined them into one dataframe

In [2]:
all_data = unemployment_rate_tojson()
print(all_data.shape)
all_data.head()

(8619382, 7)


,series_id,year,period,value,footnote_codes,area_code,measure_code
0,LAUBS060000000000003,2000,M01,5.2,,BS0600000000000,03
1,LAUBS060000000000003,2000,M02,5.3,,BS0600000000000,03
2,LAUBS060000000000003,2000,M03,5.4,,BS0600000000000,03
3,LAUBS060000000000003,2000,M04,4.7,,BS0600000000000,03
4,LAUBS060000000000003,2000,M05,4.6,,BS0600000000000,03


In [3]:
# look at the missing values
all_data.isnull().sum()

series_id         0
year              5
period            5
value             5
footnote_codes    5
area_code         0
measure_code      0
dtype: int64

In [4]:
# since all 5 records with missing values contain no information at all, I will drop them from the data
all_data.loc[all_data["year"].isnull() == True, :]

,series_id,year,period,value,footnote_codes,area_code,measure_code
2117115,,None,None,None,None,,
4234751,,None,None,None,None,,
6390139,,None,None,None,None,,
8552287,,None,None,None,None,,
8619381,,None,None,None,None,,


In [5]:
all_data = all_data.dropna()
all_data.isnull().sum()

series_id         0
year              0
period            0
value             0
footnote_codes    0
area_code         0
measure_code      0
dtype: int64

In [6]:
# extract the month numbers from "period" field
all_data["month"] = all_data["period"].apply(lambda x: x.strip("M"))
# filter records that are unemployment rate measures (measure_code == "03")
all_data = all_data.loc[all_data["measure_code"] == "03", :]

### match county names using area_code

In [7]:
area_url = "https://download.bls.gov/pub/time.series/la/la.area"
area_df = crawl_data(area_url)
# extract only county level area from area_df ("area_type_code" == "F")
county_df = area_df.loc[area_df["area_type_code"] == "F", :]
# match county names to all_data using county_df, 
# filter out records that are not county level, and month == "13" (which is the average unemployment rate of the year)
all_data = all_data.merge(county_df[["area_code", "area_text"]], on="area_code", how="left").fillna(0)
all_data = all_data.loc[(all_data["area_text"] != 0)&(all_data["month"] != "13"), ["year", "month", "value", "area_text"]]
all_data.head()

,year,month,value,area_text
12025,2000,01,4.4,"Autauga County, AL"
12026,2000,02,4.5,"Autauga County, AL"
12027,2000,03,3.9,"Autauga County, AL"
12028,2000,04,3.1,"Autauga County, AL"
12029,2000,05,3.5,"Autauga County, AL"


In [8]:
n = all_data["area_text"].nunique()
print(f"There is {n} unique counties in the dataset.")

There is 3222 unique counties in the dataset.


### There are 70 rows that has no unemployment value

In [9]:
all_data.loc[all_data["value"] == "-", :]

,year,month,value,area_text
612503,2005,09,-,"Jefferson Parish, LA"
612504,2005,10,-,"Jefferson Parish, LA"
612505,2005,11,-,"Jefferson Parish, LA"
612506,2005,12,-,"Jefferson Parish, LA"
612508,2006,01,-,"Jefferson Parish, LA"
...,...,...,...,...
614199,2006,02,-,"St. Tammany Parish, LA"
614200,2006,03,-,"St. Tammany Parish, LA"
614201,2006,04,-,"St. Tammany Parish, LA"
614202,2006,05,-,"St. Tammany Parish, LA"


In [10]:
# These counties and the number of records that are missing are shown below
all_data.loc[all_data["value"] == "-", "area_text"].value_counts()

Jefferson Parish, LA               10
St. Charles Parish, LA             10
St. John the Baptist Parish, LA    10
Orleans Parish, LA                 10
St. Tammany Parish, LA             10
St. Bernard Parish, LA             10
Plaquemines Parish, LA             10
Name: area_text, dtype: int64

### change data format to required format

In [11]:
us_state = {'AL': 'Alabama',
        'AK': 'Alaska',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'AR': 'Arkansas',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DE': 'Delaware',
        'DC': 'District of Columbia',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'IA': 'Iowa',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'ME': 'Maine',
        'MD': 'Maryland',
        'MA': 'Massachusetts',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MS': 'Mississippi',
        'MO': 'Missouri',
        'MT': 'Montana',
        'NE': 'Nebraska',
        'NV': 'Nevada',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NY': 'New York',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'MP': 'Northern Mariana Islands',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VT': 'Vermont',
        'VI': 'Virgin Islands',
        'VA': 'Virginia',
        'WA': 'Washington',
        'WV': 'West Virginia',
        'WI': 'Wisconsin',
        'WY': 'Wyoming'}
# change the format to match data in MongoDB
all_data["date"] = pd.to_datetime(all_data["year"] + all_data["month"], format="%Y%m")
all_data = all_data[["area_text", "date", "value"]].sort_values(["area_text", "date"]).rename(columns={"value": "unemployment_rate"})
# create a dataframe "county_level" that represents one county as a row
county_level = all_data["area_text"].value_counts().reset_index().rename(columns={"index": "area_text", "area_text": "count"})

# extract the FirstDate of each county
tmp = all_data.groupby("area_text")["date"].first().reset_index().rename(columns={"date": "StartDate"})
county_level = county_level.merge(tmp, on="area_text")
# extract the EndDate of each county
tmp = all_data.groupby("area_text")["date"].last().reset_index().rename(columns={"date": "EndDate"})
county_level = county_level.merge(tmp, on="area_text")
# put all unemployment rates of a county into a list
tmp = all_data.groupby("area_text")["unemployment_rate"].agg(lambda x: x.values.tolist()).reset_index()
county_level = county_level.merge(tmp, on="area_text")

# split area_text into state and county
county_level["state"] = county_level["area_text"].apply(lambda x: split_string(x)).map(us_state)
county_level["state"] = county_level["state"].fillna(county_level["area_text"])
county_level["county"] = county_level["area_text"].apply(lambda x: x.split(",")[0].strip())
# select only the columns needed and transform them into json 
county_level = county_level[["county", "state", "StartDate", "EndDate", "unemployment_rate"]]
county_level["StartDate"] = county_level["StartDate"].dt.strftime('%Y-%m-%d')
county_level["EndDate"] = county_level["EndDate"].dt.strftime('%Y-%m-%d')
county_level.head()

,county,state,StartDate,EndDate,unemployment_rate
0,Los Angeles County,California,2000-01-01,2020-03-01,"[5.9, 5.7, 5.5, 5.2, 5.3, 5.4, 5.9, 5.8, 5.3, ..."
1,District of Columbia,District of Columbia,2000-01-01,2020-03-01,"[5.9, 5.7, 5.5, 4.7, 5.0, 5.6, 5.8, 5.7, 5.9, ..."
2,Miami-Dade County,Florida,2000-01-01,2020-03-01,"[4.1, 4.2, 4.5, 4.0, 4.3, 4.4, 4.6, 4.8, 4.5, ..."
3,Wilkes County,Georgia,2000-01-01,2020-02-01,"[4.8, 4.4, 4.8, 5.4, 4.7, 4.7, 4.9, 4.8, 4.4, ..."
4,Saline County,Nebraska,2000-01-01,2020-02-01,"[2.9, 2.8, 2.5, 2.5, 2.7, 3.2, 3.0, 2.5, 2.4, ..."


In [12]:
county_level.shape

(3222, 5)

In [13]:
s = StringIO(county_level.to_json(orient='records'))
json_ls = json.load(s)